# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [15]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from collections import defaultdict
from sqlalchemy import create_engine
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
import seaborn as sns
%matplotlib inline

import pickle

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression, Perceptron
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, precision_score
from sklearn.metrics import recall_score, make_scorer
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from imblearn.over_sampling import SMOTE, SVMSMOTE, BorderlineSMOTE
from imblearn.over_sampling import RandomOverSampler
import imblearn.pipeline as imb_pipe

[nltk_data] Downloading package punkt to /home/brunom/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/brunom/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/brunom/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [16]:
pd.options.display.max_rows = 500
pd.options.display.max_columns = 500

In [17]:
# load data from database
engine = create_engine('sqlite:///data/disasterResponse.db')
df = pd.read_sql('SELECT * FROM messages_categories', engine)

In [18]:
df.head()

,id,message,original,genre,aid_centers,aid_related,buildings,child_alone,clothing,cold,death,direct_report,earthquake,electricity,fire,floods,food,hospitals,infrastructure_related,medical_help,medical_products,military,missing_people,money,offer,other_aid,other_infrastructure,other_weather,refugees,related,request,search_and_rescue,security,shelter,shops,storm,tools,transport,water,weather_related
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,1
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,0,1,1,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [19]:
df.iloc[:,4:].columns.values

array(['aid_centers', 'aid_related', 'buildings', 'child_alone',
       'clothing', 'cold', 'death', 'direct_report', 'earthquake',
       'electricity', 'fire', 'floods', 'food', 'hospitals',
       'infrastructure_related', 'medical_help', 'medical_products',
       'military', 'missing_people', 'money', 'offer', 'other_aid',
       'other_infrastructure', 'other_weather', 'refugees', 'related',
       'request', 'search_and_rescue', 'security', 'shelter', 'shops',
       'storm', 'tools', 'transport', 'water', 'weather_related'],
      dtype=object)

In [20]:
# preliminary cleaning
df.iloc[:,4:] = df.iloc[:,4:].astype('int64')
df.drop(['related','child_alone'], axis=1, inplace=True)

### 2. Write a tokenization function to process your text data

In [21]:
stop_words = set(stopwords.words('english'))
def tokenize(sentence):
    stemmer = PorterStemmer()
    lemmatizer = WordNetLemmatizer()
    sentence = re.sub("[^a-zA-Z]", " ", sentence)
    return [lemmatizer.lemmatize(word).lower().strip() for word in nltk.word_tokenize(sentence)\
            if not word in stop_words] 

message = df.message.apply(lambda x: " ".join(tokenize(x)))

In [22]:
message[:10][0]

'weather update cold front cuba could pas haiti'

#### Classes are very imbalanced

In [23]:
print(df.shape)
df.iloc[:,4:].apply(lambda x: x.value_counts())

(26216, 38)


,aid_centers,aid_related,buildings,clothing,cold,death,direct_report,earthquake,electricity,fire,floods,food,hospitals,infrastructure_related,medical_help,medical_products,military,missing_people,money,offer,other_aid,other_infrastructure,other_weather,refugees,request,search_and_rescue,security,shelter,shops,storm,tools,transport,water,weather_related
0,25907,15356,24883,25811,25686,25022,21141,23761,25684,25934,24061,23293,25933,24511,24132,24903,25356,25918,25612,26098,22770,25065,24840,25341,21742,25492,25745,23902,26096,23773,26057,25015,24544,18919
1,309,10860,1333,405,530,1194,5075,2455,532,282,2155,2923,283,1705,2084,1313,860,298,604,118,3446,1151,1376,875,4474,724,471,2314,120,2443,159,1201,1672,7297


In [24]:
df.isnull().sum()

id                            0
message                       0
original                  16046
genre                         0
aid_centers                   0
aid_related                   0
buildings                     0
clothing                      0
cold                          0
death                         0
direct_report                 0
earthquake                    0
electricity                   0
fire                          0
floods                        0
food                          0
hospitals                     0
infrastructure_related        0
medical_help                  0
medical_products              0
military                      0
missing_people                0
money                         0
offer                         0
other_aid                     0
other_infrastructure          0
other_weather                 0
refugees                      0
request                       0
search_and_rescue             0
security                      0
shelter 

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [25]:
class Clean_text(BaseEstimator, TransformerMixin):
    def __init__(self):
        return None
        
    def fit(self, X=None, y=None):
        return self
    
    def transform(self, X, y=None):
        return pd.Series(X).apply(lambda x: " ".join(self.tokenize(x))).values
        #return X, y
    
    def tokenize(self, sentence):
        #stemmer = PorterStemmer()
        lemmatizer = WordNetLemmatizer()
        sentence = re.sub("[^a-zA-Z]", " ", sentence)
        return [lemmatizer.lemmatize(word).lower().strip() for word in nltk.word_tokenize(sentence)\
            if not word in stop_words] 

In [26]:
clean_text = Clean_text()
clean_text.transform(df.message)[:5]

array(['weather update cold front cuba could pas haiti', 'is hurricane',
       'looking someone name',
       'un report leogane destroyed only hospital st croix functioning needs supply desperately',
       'say west side haiti rest country today tonight'], dtype=object)

In [27]:
df_message_test = clean_text.transform(df.message)
countVect = CountVectorizer(min_df=10,  stop_words="english", ngram_range=(1, 2))
df_message_vect = countVect.fit_transform(df_message_test)
countVect.get_feature_names()[:20]

['ababa',
 'abandon',
 'abandoned',
 'ability',
 'able',
 'abroad',
 'absence',
 'abundant',
 'abuse',
 'accelerated',
 'accept',
 'accepted',
 'accepting',
 'access',
 'access clean',
 'access safe',
 'accessible',
 'accident',
 'accommodate',
 'accommodation']

In [29]:
ml_pipeline = Pipeline([
    ('clean_text', Clean_text()),
    ('vectorizer', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('abclf', MultiOutputClassifier(AdaBoostClassifier()))])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [30]:
X = df.message.copy()
y = df.iloc[:,4:].values 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=32) # training
X_train.shape, y_train.shape
ml_pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('clean_text', Clean_text()), ('vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=...or=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=None))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

#### Because classes are imbalanced, accuracy is not a good metric for this problem.

In [31]:
X_train.head()

23313    Forty-eight teams of medical and paramedical s...
7683                   NOTES: this message is not complete
23462    All graphs provide evidence that, considering ...
22334    construction material: bricks - 500 t; cement ...
6522     I can arrange shelter shelter, i live out in t...
Name: message, dtype: object

In [32]:
def cross_val(X_train, y_train, pipeline):
    labels = []
    preds = []
    kf = KFold(n_splits=5, shuffle=True)
    for train_indices, test_indices in kf.split(X_train, y_train):
   
        X_train_kf = X_train.iloc[train_indices]
        y_train_kf = y_train[train_indices]
    
        X_test_kf = X_train.iloc[test_indices]
        y_test_kf = y_train[test_indices]
  
        pipeline.fit(X_train_kf, y_train_kf)
    
                
        labels.extend(y_test_kf)
        preds.extend(pipeline.predict(X_test_kf))

    return labels, preds

In [33]:
labels, preds = cross_val(X_train, y_train, ml_pipeline)

In [34]:
def predict_scores(labels, preds):
    list_scorings = [f1_score, accuracy_score, precision_score, recall_score]
    labels_df = pd.DataFrame(labels)
    preds_df = pd.DataFrame(preds)
    return pd.DataFrame({scoring.__name__: {column: scoring(labels_df.iloc[:,n], preds_df.iloc[:,n]) 
                                            for n, column in enumerate(df.iloc[:,4:].columns.values)} 
                         for scoring in list_scorings})

In [35]:
scoring_df_original = predict_scores(labels, preds)

In [56]:
scoring_df_original.reset_index()

,index,f1_score,accuracy_score,precision_score,recall_score
0,aid_centers,0.113208,0.986553,0.260870,0.072289
1,aid_related,0.676056,0.758726,0.756664,0.610970
2,buildings,0.495606,0.958945,0.653787,0.399057
3,clothing,0.581006,0.989271,0.728972,0.482972
4,cold,0.447489,0.982691,0.653333,0.340278
5,death,0.555483,0.967337,0.715719,0.453871
6,direct_report,0.511872,0.849037,0.692821,0.405868
7,earthquake,0.827236,0.969149,0.880114,0.780353
8,electricity,0.370607,0.981213,0.591837,0.269767
9,fire,0.317073,0.989319,0.530612,0.226087


In [57]:
scoring_df_original.to_csv('scoring.csv')

In [66]:
df2 = pd.read_csv('scoring.csv')
df2.head()

,Unnamed: 0,f1_score,accuracy_score,precision_score,recall_score
0,aid_centers,0.113208,0.986553,0.260870,0.072289
1,aid_related,0.676056,0.758726,0.756664,0.610970
2,buildings,0.495606,0.958945,0.653787,0.399057
3,clothing,0.581006,0.989271,0.728972,0.482972
4,cold,0.447489,0.982691,0.653333,0.340278


In [38]:
# sanity check
X_mod = df.message.copy()
y_mod = df.iloc[:,4].values 

ml_pipeline_single = Pipeline([
    ('clean_text', Clean_text()),
    ('vectorizer', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('abclf', AdaBoostClassifier())])

X_train_mod, X_test_mod, y_train_mod, y_test_mod = train_test_split(X_mod, y_mod, test_size=0.2, random_state=32)
ml_pipeline_single.fit(X_train_mod, y_train_mod)
labels_df_mod, preds_df_mod = cross_val(X_train_mod, y_train_mod, ml_pipeline_single)
f1_score(labels_df_mod, preds_df_mod)

0.07028753993610223

In [60]:
def train(X, y, model, columns):
    # train test split
    X_train, X_test, y_train, y_test =\
    train_test_split(X, y, test_size=0.2, random_state=32)

    # fit model
    model.fit(X_train, y_train)

    # output model test results
    list_scorings = [f1_score, precision_score, recall_score]
    labels_df = pd.DataFrame(y_test)
    preds_df = pd.DataFrame(model.predict(X_test))
    return pd.DataFrame({scoring.__name__: {column: scoring(labels_df.iloc[:,n], preds_df.iloc[:,n], pos_label=1) for n, column in enumerate(columns)} for scoring in list_scorings})


In [63]:
model = Pipeline([
    ('clean_text', Clean_text()),
    ('vectorizer', CountVectorizer(min_df= 5,stop_words="english", ngram_range=(1, 2))),
    ('tfidf', TfidfTransformer(sublinear_tf=True, norm='l2')),
    ('abclf', MultiOutputClassifier(AdaBoostClassifier(random_state=34, n_estimators=100, learning_rate=1.0)))])
columns = df.iloc[:,4:].columns.values

score_df = train(X, y, model, columns)

In [64]:
score_df

,f1_score,precision_score,recall_score
aid_centers,0.095238,0.166667,0.066667
aid_related,0.687308,0.794675,0.605500
buildings,0.522484,0.628866,0.446886
clothing,0.446154,0.604167,0.353659
cold,0.376812,0.650000,0.265306
death,0.518135,0.740741,0.398406
direct_report,0.545003,0.701278,0.445685
earthquake,0.837054,0.880282,0.797872
electricity,0.341463,0.451613,0.274510
fire,0.202899,0.411765,0.134615


### 6. Improve your model
Use grid search to find better parameters. 

In [39]:
ml_pipeline.get_params()

{'memory': None,
 'steps': [('clean_text', Clean_text()),
  ('vectorizer',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=None, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('abclf',
   MultiOutputClassifier(estimator=AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
             learning_rate=1.0, n_estimators=50, random_state=None),
              n_jobs=None))],
 'clean_text': Clean_text(),
 'vectorizer': CountVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
         lowercase=True, max_df=1.0, max

In [59]:
X = df.message.copy()
y = df.iloc[:,4:].values 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=32) 

#parameters = {
#    'vectorizer__min_df': [5,10,20],
#    'vectorizer__ngram_range': [(1,1),(1,2)],
#    'abclf__estimator__learning_rate': [0.1,0.2,0.3,1.0],
#    'abclf__estimator__n_estimators': [50, 100]
#}

parameters = {
    'vectorizer__min_df': [5,10],
    'vectorizer__ngram_range': [(1,1),(1,2)],
    'tfidf__norm': ['l2','l1'],
    'tfidf__sublinear_tf': [True, False],
    'abclf__estimator__learning_rate': [0.1,0.2,0.3,1.0],
    'abclf__estimator__n_estimators': [50,100]
}

f1_scorer = make_scorer(f1_score, average='macro')
grid_clf = GridSearchCV(ml_pipeline, param_grid=parameters, cv=5, verbose=10)

grid_clf.fit(X_train, y_train)

/home/brunom/.miniconda2/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Fitting 3 folds for each of 128 candidates, totalling 384 fits
[CV] abclf__estimator__learning_rate=0.1, abclf__estimator__n_estimators=50, tfidf__norm=l2, tfidf__sublinear_tf=True, vectorizer__min_df=5, vectorizer__ngram_range=(1, 1) 
[CV]  abclf__estimator__learning_rate=0.1, abclf__estimator__n_estimators=50, tfidf__norm=l2, tfidf__sublinear_tf=True, vectorizer__min_df=5, vectorizer__ngram_range=(1, 1), score=0.43355743098269206, total=  55.2s
[CV] abclf__estimator__learning_rate=0.1, abclf__estimator__n_estimators=50, tfidf__norm=l2, tfidf__sublinear_tf=True, vectorizer__min_df=5, vectorizer__ngram_range=(1, 1) 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.0min remaining:    0.0s


[CV]  abclf__estimator__learning_rate=0.1, abclf__estimator__n_estimators=50, tfidf__norm=l2, tfidf__sublinear_tf=True, vectorizer__min_df=5, vectorizer__ngram_range=(1, 1), score=0.4316978972965241, total=  55.1s
[CV] abclf__estimator__learning_rate=0.1, abclf__estimator__n_estimators=50, tfidf__norm=l2, tfidf__sublinear_tf=True, vectorizer__min_df=5, vectorizer__ngram_range=(1, 1) 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  2.1min remaining:    0.0s


...

[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:  8.7min remaining:    0.0s


[CV]  abclf__estimator__learning_rate=0.1, abclf__estimator__n_estimators=50, tfidf__norm=l2, tfidf__sublinear_tf=True, vectorizer__min_df=10, vectorizer__ngram_range=(1, 1), score=0.4361945636623748, total=  56.1s
[CV] abclf__estimator__learning_rate=0.1, abclf__estimator__n_estimators=50, tfidf__norm=l2, tfidf__sublinear_tf=True, vectorizer__min_df=10, vectorizer__ngram_range=(1, 2) 


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  9.7min remaining:    0.0s


[CV]  abclf__estimator__learning_rate=0.1, abclf__estimator__n_estimators=50, tfidf__norm=l2, tfidf__sublinear_tf=True, vectorizer__min_df=10, vectorizer__ngram_range=(1, 2), score=0.43026748676870263, total=  55.6s
[CV] abclf__estimator__learning_rate=0.1, abclf__estimator__n_estimators=50, tfidf__norm=l2, tfidf__sublinear_tf=True, vectorizer__min_df=10, vectorizer__ngram_range=(1, 2) 
[CV]  abclf__estimator__learning_rate=0.1, abclf__estimator__n_estimators=50, tfidf__norm=l2, tfidf__sublinear_tf=True, vectorizer__min_df=10, vectorizer__ngram_range=(1, 2), score=0.43026748676870263, total=  55.7s
[CV] abclf__estimator__learning_rate=0.1, abclf__estimator__n_estimators=50, tfidf__norm=l2, tfidf__sublinear_tf=True, vectorizer__min_df=10, vectorizer__ngram_range=(1, 2) 
[CV]  abclf__estimator__learning_rate=0.1, abclf__estimator__n_estimators=50, tfidf__norm=l2, tfidf__sublinear_tf=True, vectorizer__min_df=10, vectorizer__ngram_range=(1, 2), score=0.43662374821173106, total=  56.2s
[CV]

KeyboardInterrupt: 

In [ ]:
ml_pipeline_best = grid_clf.best_estimator_
print(ml_pipeline_best)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model. Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
y_pred = ml_pipeline_best.predict(X_test)
{column: f1_score(y_test[:,n], y_pred[:,n]) for n, column in enumerate(df.iloc[:,4:].columns.values)}

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### Option 1: Oversampling with SMOTE

In [ ]:
pipeline_smote = imb_pipe.make_pipeline(Clean_text(), CountVectorizer(), SMOTE(), TfidfTransformer(),
                                        AdaBoostClassifier())

In [ ]:
y_test.shape[0], y_test.shape[1]

In [ ]:
preds = np.empty((y_test.shape[0], y_test.shape[1]))
for n, column in enumerate(df.iloc[:,4:].columns.values):
    pipeline_smote.fit(X_train, y_train[:,n])
    preds[:,n] = pipeline_smote.predict(X_test)
     
#scoring_df_original = predict_scores(y_test, preds)

In [ ]:
scoring_df_smote = predict_scores(y_test, preds)

In [ ]:
comp_df = pd.DataFrame(scoring_df_original.f1_score)
comp_df['smote'] = scoring_df_smote['f1_score'].copy()
comp_df

#### No improvement

#### Option 2: Using stemmer and more advanced options for count_vectorizer

In [ ]:
class Clean_text_with_stemmer(BaseEstimator, TransformerMixin):
    def __init__(self):
        return None
        
    def fit(self, X=None, y=None):
        return self
    
    def transform(self, X, y=None):
        
        return pd.Series(X).apply(lambda x: " ".join(self.tokenize(x))).values
    
    def tokenize(self, sentence):
        stemmer = PorterStemmer()
        lemmatizer = WordNetLemmatizer()
        sentence = re.sub("[^a-zA-Z]", " ", sentence)
        return [stemmer.stem(lemmatizer.lemmatize(word).lower().strip()) for word in nltk.word_tokenize(sentence)\
            if not word in stop_words] 

In [ ]:
# baseline - logistic
ml_pipeline_logistic = Pipeline([
    ('clean_text', Clean_text_with_stemmer()),
    ('vectorizer', CountVectorizer(min_df= 5,stop_words="english", ngram_range=(1, 2))),
    ('tfidf', TfidfTransformer(sublinear_tf=True, norm='l2')),
    ('mbclf', MultiOutputClassifier(LogisticRegression(random_state=34, solver='lbfgs')))])

In [ ]:
labels, preds = cross_val(X_train, y_train, ml_pipeline_logistic)
scoring_df_logistic = predict_scores(labels, preds)

In [ ]:
comp_df['logistic'] = scoring_df_logistic['f1_score']

In [ ]:
comp_df

In [ ]:
ml_pipeline_svc = Pipeline([
    ('clean_text', Clean_text_with_stemmer()),
    ('vectorizer', CountVectorizer(min_df= 2,stop_words="english", ngram_range=(1, 2))),
    ('tfidf', TfidfTransformer(sublinear_tf=True, norm='l2')),
    ('mbclf', MultiOutputClassifier(SVC(C=1.0, gamma=0.1)))])

In [ ]:
ml_pipeline_svc.fit(X_train, y_train)
scoring_df_svc = predict_scores(y_test, ml_pipeline_svc.predict(X_test))
comp_df['svc'] = scoring_df_svc['f1_score']
comp_df

In [ ]:
# model with count vectorizer only, no stemmer, and optimized AdaBoost
ml_pipeline = Pipeline([
    ('clean_text', Clean_text()),
    ('vectorizer', CountVectorizer(min_df = 10,stop_words="english", ngram_range=(1, 2))),
    ('mbclf', MultiOutputClassifier(AdaBoostClassifier(random_state=34, n_estimators=100, learning_rate=1.0)))])

ml_pipeline.fit(X_train, y_train)
scoring_df = predict_scores(y_test, ml_pipeline.predict(X_test))
comp_df['vect_no_stem'] = scoring_df['f1_score']
comp_df

In [ ]:
# model with count vectorizer only and optimized AdaBoost
ml_pipeline = Pipeline([
    ('clean_text', Clean_text_with_stemmer()),
    ('vectorizer', CountVectorizer(min_df = 10,stop_words="english", ngram_range=(1, 2))),
    ('mbclf', MultiOutputClassifier(AdaBoostClassifier(random_state=34, n_estimators=100, learning_rate=1.0)))])

ml_pipeline.fit(X_train, y_train)
scoring_df = predict_scores(y_test, ml_pipeline.predict(X_test))
comp_df['vect_only'] = scoring_df['f1_score']
comp_df

In [ ]:
# model with count vectorizer and tfidf and optimized AdaBoost
ml_pipeline = Pipeline([
    ('clean_text', Clean_text_with_stemmer()),
    ('vectorizer', CountVectorizer(min_df = 10,stop_words="english", ngram_range=(1, 2))),
    ('tfidf', TfidfTransformer(sublinear_tf=True, norm='l1')),
    ('mbclf', MultiOutputClassifier(AdaBoostClassifier(random_state=34, n_estimators=100, learning_rate=1.0)))])

ml_pipeline.fit(X_train, y_train)
scoring_df = predict_scores(y_test, ml_pipeline.predict(X_test))
comp_df['vect_tfidf'] = scoring_df['f1_score']
comp_df

In [ ]:
# final model with optimized AdaBoost
ml_pipeline_best = Pipeline([
    ('clean_text', Clean_text_with_stemmer()),
    ('vectorizer', CountVectorizer(min_df = 10,stop_words="english", ngram_range=(1, 2))),
    ('tfidf', TfidfTransformer(sublinear_tf=True, norm='l1')),
    ('mbclf', MultiOutputClassifier(AdaBoostClassifier(random_state=34, n_estimators=100, learning_rate=1.0)))])

In [ ]:
ml_pipeline_best.fit(X_train, y_train)

In [ ]:
scoring_df_best = predict_scores(y_test, ml_pipeline_best.predict(X_test))
comp_df['best'] = scoring_df_best['f1_score']
comp_df

In [ ]:
pipeline_smote = imb_pipe.make_pipeline(Clean_text_with_stemmer(), 
                                        CountVectorizer(min_df = 10,stop_words="english", ngram_range=(1, 2)), 
                                        TfidfTransformer(sublinear_tf=True, norm='l1'), SMOTE(), 
                                        AdaBoostClassifier(random_state=34, n_estimators=100, learning_rate=0.2))

preds = np.empty((y_test.shape[0], y_test.shape[1]))
for n, column in enumerate(df.iloc[:,4:].columns.values):
    pipeline_smote.fit(X_train, y_train[:,n])
    preds[:,n] = pipeline_smote.predict(X_test)

scoring_df_smote = predict_scores(y_test, preds)
comp_df['smote_2'] = scoring_df_smote['f1_score'].copy()
comp_df

In [ ]:
df_tools = df[df.tools==1].copy()
df_tools['prediction'] = pipeline_smote.predict(df_tools.message)

In [ ]:
df_tools.message.iloc[0]
pred_proba = ml_pipeline_best.predict_proba(df_tools.message.iloc[0])
#sns.barplot(x="day", y="total_bill", data=tips)
df2 = pd.melt(pd.DataFrame({i:j[:,1] for i,j in zip(df.iloc[:,4:].columns, pred_proba)}))
print(df2.head())
sns.barplot(x='value', y='variable', data=df2, color='blue')

In [ ]:
df_tools[['message','prediction']] # the class tools is not well-defined

### 9. Export your model as a pickle file

In [ ]:


filename='data/ml_pipeline_best.pkl'
pickle.dump(ml_pipeline_best, open(filename,'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

## Code Bank

In [ ]:
# beautiful solution
df.message.apply(lambda x: " ".join([stemmer.stem(word) for word in re.sub("[^a-zA-Z]", " ", x).split() 
          if word not in stop_words]) )

In [ ]:
class Select_features(BaseEstimator, TransformerMixin):
    # constructor
    def __init__(self, column_names):
        self.column_names = column_names 
    
    # pipeline must implement fit
    def fit(self, X=None, y=None):
        return self
    
    # return new dataframe with selected features
    def transform(self, X=None, y=None):
        return X[self.column_names] 

In [ ]:
X = vector_features
y = MultiLabelBinarizer().fit_transform(labels)
print(len(labels))
print(y.shape)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=32)

vectorizer = CountVectorizer(min_df= 5,stop_words="english", ngram_range=(1, 2))
clf = OneVsRestClassifier(BernoulliNB())
message_pipeline = Pipeline([('vectorizer', vectorizer),('clf',clf)])
message_pipeline.fit()
message_pipeline.named_steps['vectorizer'].transform([df_clean_message])